<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/RedditConversationQuality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Reddit Data  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Pushshift API

In [19]:
base_path = "/content/drive/MyDrive/University/RedditData/"

##Functions

In [20]:
import json

def filter_json_objects(data, keys):
    filtered_data = []
    for item in data:
        filtered_item = {}
        for key in keys:
            if key in item:
                filtered_item[key] = item[key]
        filtered_data.append(filtered_item)
    return filtered_data

In [21]:
def write_last_timestamp(last_timestamp, entry):
  with open(f"{base_path}last_timestamp_{entry}.txt", "w") as f:
    f.write(str(last_timestamp))

In [22]:
def read_last_timestamp(default, entry):
  try:
    with open(f"{base_path}last_timestamp_{entry}.txt", "r") as f:
        return int(f.read())
  except FileNotFoundError:
    return default

In [23]:
def write_data_to_file(data, entry):
  with open(f"{base_path}{entry}s.json", "a") as f:
    for element in data:
        json.dump(element, f)
        f.write("\n")

In [67]:
import requests
import datetime
import json
import time

def extract_date_based_data_from_reddit(entry_type, keys, start_date, end_date, step):
  # type can be either submission or comment
  # keys are fields from submission or comment that we want to save
  # start date and end date shows the date interval which data is collected


  # read the last UTC timestamp from file
  last_timestamp = read_last_timestamp(int(end_date.timestamp()), entry_type)

  # define the API endpoint
  const_url = f"https://api.pushshift.io/reddit/search/{entry_type}/?size=500&filter={','.join(keys)}&sort=created_utc&"
  url_template = const_url + "after={}&before={}"
  # get the set of enteries using pagination
  while True:
    new_url = url_template.format(
        max(last_timestamp-step,int(start_date.timestamp())),
         last_timestamp)
    try:
        new_response = requests.get(new_url)
        enteries = new_response.json()["data"]
        write_data_to_file(enteries, entry_type)
        # save the UTC timestamp of the last entry to file
        print(len(enteries))
        if len(enteries) == 0:
          last_timestamp -= step
          continue
        last_entry = enteries[-1]
        last_timestamp = last_entry["created_utc"]
        last_entry_date = datetime.datetime.fromtimestamp(last_timestamp)
        print(last_entry_date)
        write_last_timestamp(last_timestamp, entry_type)
        if last_timestamp == int(start_date.timestamp()):
          break
    except json.JSONDecodeError or requests.exceptions.HTTPError:
        if new_response.status_code == 429:
            retry_after = 30#int(new_response.headers.get("Retry-After"))
            print(f"Got HTTP error 429, waiting {retry_after} seconds and retrying...")
            time.sleep(retry_after)
            continue
        else:
            raise e

##Write submmisons to file

In [ ]:
# define the date range
start_date = datetime.datetime(2023, 3, 1)
end_date = datetime.datetime(2023, 3, 31)
keys = ['created_utc', 'id']

extract_date_based_data_from_reddit("submission", keys, start_date, end_date, 10)

##Write sample submission fields to file

In [ ]:
import requests
import json

# define the API endpoint with the sample submission ID
submission_id = "1273r9g"
url = f"https://api.pushshift.io/reddit/submission/search/?ids={submission_id}"

# make the API request and get the submission
response = requests.get(url)
submission_list = response.json()["data"]

# get the first submission from the list
submission = submission_list[0]

# write the submission keys to a file
with open("submission_keys.txt", "w") as f:
    for key in submission.keys():
        f.write(key + "\n")

##Write comments to file

###With Pushshift date search

In [ ]:
# define the date range
start_date = datetime.datetime(2023, 3, 1)
end_date = datetime.datetime(2023, 3, 31)
keys = ['created_utc', 'id']

extract_date_based_data_from_reddit("comment", keys, start_date, end_date, 2)